In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM4_ESM.xlsx', sheet_name='1J')

In [4]:
grp1, grp2, grp3 = data_df['HFD'].dropna().values, data_df['HFD+SAL'].dropna().values, data_df['HFD+SANA'].dropna().values

In [5]:
collect = [grp1, grp2, grp3]

In [6]:
len(np.concatenate([grp1, grp2, grp3]))

24

In [7]:
scipy.stats.f_oneway(grp1, grp2, grp3)

F_onewayResult(statistic=15.127670752029875, pvalue=8.531820585519195e-05)

In [8]:
comp = multicomp.MultiComparison(data=np.concatenate([grp1, grp2, grp3]), 
                                  groups=np.concatenate([['HFD']*len(grp1), 
                                                         ['HFD+SAL']*len(grp2), 
                                                         ['HFD+SANA']*len(grp3)]))
tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
tbl

group1,group2,stat,pval,pval_corr,reject
HFD,HFD+SAL,-1.7969,0.094,0.2819,False
HFD,HFD+SANA,3.0714,0.0083,0.0249,True
HFD+SAL,HFD+SANA,6.7789,0.0,0.0,True


In [13]:
a1

(array([[-1.79686067e+00,  9.39579489e-02],
        [ 3.07142529e+00,  8.28967390e-03],
        [ 6.77889490e+00,  8.89453119e-06]]),
 array([False,  True,  True]),
 array([2.81873847e-01, 2.48690217e-02, 2.66835936e-05]),
 0.016952427508441503,
 0.016666666666666666)

In [14]:
scipy.stats.tukey_hsd(*[grp1, grp2, grp3]).pvalue

array([[1.00000000e+00, 2.01905784e-01, 4.38144328e-03],
       [2.01905784e-01, 1.00000000e+00, 6.77500255e-05],
       [4.38144328e-03, 6.77500255e-05, 1.00000000e+00]])

Here is Figure 1J:

![file](../img/pub_fig_1j.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38) for this figure, we arrive at F(2, 21) = 15.1, p = 8.53E-5, which supports the authors' statement that treatment did not affect digested energy. However, the p-values for the post-hoc test appear to be wrong for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test). The p-values shown here instead match the expected output of Tukey's post-hoc test, although with some inconsistencies in rounding.

| Comparison | Published p-value | Bonferroni p-value | Tukey p-value |
| -- | --- | --- | --- |
| HFD vs HFD+SANA | 0.0044 | 0.0249 | 0.00438 |
| HFD vs HFD+SAL | 0.2019 | 0.282 | 0.2019 |
|  HFD+SAL vs HFD+SANA | 0.00008|||

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).